In [1]:
import pandas as pd
from nltk import FreqDist
import pickle
import numpy as np

In [2]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [6]:
LAGT = pd.read_json("../data/large_data/LAGT_metadata.json")

In [7]:
LAGT.head(5)

,author_id,doc_id,GLAUx?,filename,subcorpus,author,title,date_avr,provenience,tlg_epithet,n_sentences,wordcount
0,tlg0001,tlg0001.tlg001,True,tlg0001.tlg001.perseus-grc2.xml,classical,Apollonius Rhodius,Argonautica,-2.5,pagan,Epici/-ae,3252,38822
1,tlg0003,tlg0003.tlg001,True,tlg0003.tlg001.perseus-grc2.xml,classical,Thucydides,The Peloponnesian War,-4.5,pagan,Historici/-ae,6068,150118
2,tlg0004,tlg0004.tlg001,False,tlg0004.tlg001.perseus-grc1.xml,roman,Diogenes Laertius,Lives of Eminent Philosophers,2.5,pagan,Biographi,10245,110763
3,tlg0005,tlg0005.tlg001,True,tlg0005.tlg001.perseus-grc1.xml,classical,Theocritus,Idylls,-3.0,pagan,Bucolici,1982,19200
4,tlg0005,tlg0005.tlg002,True,tlg0005.tlg002.perseus-grc1.xml,classical,Theocritus,Epigrams,-3.0,pagan,Bucolici,152,1734


# Prepare subcorpora

In [9]:
def assign_subcorpus(row):
    if row["date_avr"] <= -5:
        subcorpus = "archaic"
    elif row["date_avr"] < -2:
        subcorpus = "classical"
    elif (row["provenience"]=="christian") & (row["date_avr"]<3):
        subcorpus = "christian"
    elif (row["provenience"]=="pagan") & (row["date_avr"]<3):
        subcorpus = "roman"
    elif (row["provenience"]=="jewish"):
        subcorpus = "jewish"
    else:
        subcorpus = None
    return subcorpus

In [10]:
LAGT["subcorpus"] = LAGT.apply(lambda row: assign_subcorpus(row), axis=1)

In [11]:
df_size = LAGT.groupby("subcorpus").size().reset_index()
df_size.rename(columns={0:"n_works"}, inplace=True)
df_size

,subcorpus,n_works
0,archaic,26
1,christian,104
2,classical,408
3,jewish,95
4,roman,596


In [12]:
df_sum = LAGT.groupby("subcorpus").sum()[["wordcount", "n_sentences", "GLAUx?"]]
pd.merge(df_size, df_sum.reset_index())

,subcorpus,n_works,wordcount,n_sentences,GLAUx?
0,archaic,26,331410,26561,24
1,christian,104,1711991,180658,19
2,classical,408,4104920,319695,370
3,jewish,95,1953311,166050,4
4,roman,596,11481179,932972,409


In [13]:
LAGT[LAGT["subcorpus"]=="archaic"]

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,n_sentences,lemmatized_sentences,GLAUx?,subcorpus
202,tlg0012.tlg001.perseus-grc2.xml,Homer,Iliad (Greek). Machine readable text,111888,tlg0012,tlg0012.tlg001,8 B.C.,-7.5,{'-7.5': 1},-7.5,pagan,Epici/-ae,"μῆνιν ἄειδε θεὰ Πηληϊάδεω Ἀχιλῆος οὐλομένην, ...",8415,"[[μῆνις, ἀείδω, θεά, Πηληϊάδης, Ἀχιλλεύς, ὄλλυ...",True,archaic
203,tlg0012.tlg002.perseus-grc2.xml,Homer,Odyssey (Greek). Machine readable text,87177,tlg0012,tlg0012.tlg002,8 B.C.,-7.5,{'-7.5': 1},-7.5,pagan,Epici/-ae,"ἄνδρα μοι ἔννεπε, μοῦσα, πολύτροπον, ὃς μάλα π...",6729,"[[ἀνήρ, ἐνέπω, Μοῦσα, πολύτροπος, πολύς, πλάζω...",True,archaic
204,tlg0012.tlg003.perseus-grc1.xml,Homer,Ἐπιγράμματα,757,tlg0012,tlg0012.tlg003,8 B.C.,-7.5,{'-7.5': 1},-7.5,pagan,Epici/-ae,"Αἰδεῖσθε ξενίων κεχρημένον ἠδὲ δόμοιο, οἳ πόλι...",72,"[[Ἕλλην, πλῆθος, Ἴλιος, στρατόω, ἑπτά, εἰμί, μ...",True,archaic
205,tlg0013.tlg001.perseus-grc2.xml,Homeric hymn,Hymn 1 To Dionysus,144,tlg0013dyon,tlg0013.tlg001,8-6 B.C.,-6.5,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"οἳ μὲν γὰρ Δρακάνῳ σʼ, οἳ δʼ Ἰκάρῳ ἠνεμοέσσῃ ...",13,"[[Δράκανος, Ἴκαρος, ἠνεμόεις, φημί, Νάξος, δῖο...",True,archaic
206,tlg0013.tlg002.perseus-grc2.xml,Homeric hymn,Hymn 2 To Demeter,3373,tlg0013deme,tlg0013.tlg002,8-6 B.C.,-6.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"Δήμητρʼ ἠύκομον, σεμνὴν θεόν, ἄρχομʼ ἀείδειν,...",256,"[[Δημήτηρ, εὔκομος, σεμνός, θεός, ἄρχω, ἀείδω,...",True,archaic
207,tlg0013.tlg003.perseus-grc2.xml,Homeric hymn,Hymn 3 To Apollo,3848,tlg0013apol,tlg0013.tlg003,8-6 B.C.,-6.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"μνήσομαι οὐδὲ λάθωμαι Ἀπόλλωνος ἑκάτοιο, ὅντε ...",295,"[[μιμνῄσκω, λανθάνω, Ἀπόλλων, ἕκατος, θεός, δῶ...",True,archaic
208,tlg0013.tlg004.perseus-grc2.xml,Homeric hymn,Hymn 4 To Hermes,4031,tlg0013herm,tlg0013.tlg004,8-6 B.C.,-5.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"Ἑρμῆν ὕμνει, Μοῦσα, Διὸς καὶ Μαιάδος υἱόν, Κυλ...",332,"[[Ἑρμῆς, ὑμνέω, Μοῦσα, Ζεύς, Μαιάς, υἱός, Κυλλ...",True,archaic
209,tlg0013.tlg005.perseus-grc2.xml,Homeric hymn,Hymn 5 To Aphrodite,2047,tlg0013aphr,tlg0013.tlg005,8-6 B.C.,-6.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"μοῦσά μοι ἔννεπε ἔργα πολυχρύσου Ἀφροδίτης, Κύ...",148,"[[Μοῦσα, ἐνέπω, ἔργον, πολύχρυσος, Ἀφροδίτη, Κ...",True,archaic
211,tlg0013.tlg007.perseus-grc2.xml,Homeric hymn,Hymn 7 To Dionysus,425,tlg0013dyo2,tlg0013.tlg007,8-6 B.C.,-5.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"ἀμφὶ Διώνυσον, Σεμέλης ἐρικυδέος υἱόν, μνήσομα...",42,"[[Διόνυσος, Σεμέλη, ἐρικυδής, υἱός, μιμνῄσκω, ...",True,archaic
223,tlg0013.tlg019.perseus-grc2.xml,Homeric hymn,Hymn 19 to Pan,336,tlg0013pan,tlg0013.tlg019,8-6 B.C.,-5.0,"{'-7.5': 0.33330000000000004, '-6.5': 0.333300...",-7.0,pagan,[],"ἀμφί μοι Ἑρμείαο φίλον γόνον ἔννεπε, Μοῦσα, αἰ...",19,"[[Ἑρμῆς, φίλος, γόνος, ἐνέπω, Μοῦσα, αἰγιπόδης...",True,archaic


In [14]:
(LAGT["subcorpus"].notnull()).sum()

1229

# Read subcorpora from ngrams - developing a memory-friendly reader

In [15]:
ids_lines = pickle.load(open("../data/ids_lines.pickle", "rb"))

In [25]:
# developing corpus reader

In [17]:
list(ids_lines.items())[:5]

[('tlg0001.tlg001', (0, 41193)),
 ('tlg0003.tlg001', (41193, 184616)),
 ('tlg0004.tlg001', (184616, 280036)),
 ('tlg0005.tlg001', (280036, 300693)),
 ('tlg0005.tlg002', (300693, 302641))]

In [33]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="archaic"]["doc_id"].tolist()
len(subcorpus_ids)

26

In [36]:
lines_list = [ ]
for id in subcorpus_ids:
    lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
ngrams_n = len(lines_list)
ngrams_n

339423

In [57]:
sample_size=1000000
if sample_size != None:
    sample_inx =np.random.randint(0, ngrams_n, sample_size)
else:
    sample_inx =[n for n in range(0, ngrams_n)]

In [58]:
sample_lines = np.array(lines_list)[sample_inx]
len(sample_lines)

1000000

In [39]:
sample_lines[:5]

array([1882202, 1882203, 1882204, 1882205, 1882206])

In [40]:
test_ngrams = []
with open("../data/large_data/corpus_ngrams_bydocid.txt", "r") as f:
    lines = f.readlines()
    for li in sample_lines:
        line = lines[li]# for line index in subcorpus lineindexes
        if bool(line.split()):
            test_ngrams.append(line.split())

In [42]:
len(test_ngrams)

339299

# Sample corpora with memory-friendly reader

In [68]:
class NgramCorpusSample(object):
    def __init__(self, ids_list, ids_lines, fname, sample_size=None, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.sample_size = sample_size
        self.ids_lines = ids_lines
        lines_list = [ ]
        for id in ids_list:
            lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
        ngrams_n = len(lines_list)
        if sample_size != None:
            sample_inx =np.random.randint(0, ngrams_n, sample_size)
        else:
            sample_inx =[n for n in range(0, ngrams_n)]
        sample_lines = np.array(lines_list)[sample_inx]
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for li in self.sample_lines:
                line = lines[li] # for line index in subcorpus lineindexes
                if bool(line.split()):
                    if (self.bow) & (self.dct != None):
                        yield self.dct.doc2bow(line.split())
                    else:
                        yield line.split()

In [75]:
subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt")

In [76]:
len(subcorpus_ngrams)

339423

In [77]:
len([doc for doc in subcorpus_ngrams])

339299

In [78]:
FreqDist([w for ng in subcorpus_ngrams for w in ng]).most_common()[:50]

[('εἰμί', 16217),
 ('ἀνήρ', 8468),
 ('πᾶς', 8442),
 ('ἔρχομαι', 7776),
 ('πολύς', 7440),
 ('θεός', 7113),
 ('ἔχω', 7058),
 ('Ζεύς', 6570),
 ('μέγας', 6206),
 ('λέγω', 5893),
 ('ναῦς', 5837),
 ('φίλος', 5082),
 ('χείρ', 4914),
 ('ὅδε', 4613),
 ('ὁράω', 4428),
 ('θυμός', 4409),
 ('φημί', 4347),
 ('υἱός', 3943),
 ('κακός', 3713),
 ('πατήρ', 3679),
 ('γίγνομαι', 3586),
 ('Ἀχαιός', 3521),
 ('Ὀδυσσεύς', 3379),
 ('Τρώς', 3290),
 ('δίδωμι', 3245),
 ('παῖς', 3210),
 ('γῆ', 3125),
 ('αὐτός', 3079),
 ('ἐμός', 3043),
 ('τίθημι', 2942),
 ('ἄνθρωπος', 2930),
 ('φέρω', 2918),
 ('βάλλω', 2874),
 ('ἔπος', 2852),
 ('ἵππος', 2809),
 ('φρήν', 2799),
 ('ἀγαθός', 2792),
 ('ἵστημι', 2719),
 ('οὗτος', 2624),
 ('αἱρέω', 2611),
 ('καλός', 2522),
 ('πόλις', 2483),
 ('πρῶτος', 2436),
 ('βαίνω', 2361),
 ('οἶδα', 2354),
 ('ἄγω', 2320),
 ('δῖος', 2300),
 ('ἐθέλω', 2276),
 ('ἑταῖρος', 2229),
 ('τις', 2194)]